In [1]:
from collections import defaultdict
import random
import numpy as np
np.set_printoptions(precision=2, suppress=True)

import time
import copy 
import multiprocess as mp

import gym
from env import FrozenLakeCustom, FrozenLakeSimulator

from mcts_haver import run_mcts_trial
from value_iteration import value_iteration

from config import parse_args

import logging
logger = logging.getLogger()
logger.setLevel(logging.FATAL)

In [2]:
np.random.seed(0)
random.seed(0)

# params
args = parse_args()
args["update_method"] = "haver"
args["rollout_method"] = "vit"


#
env_id = "FrozenLake-v1"
env = FrozenLakeCustom(
    map_name=args["map_name"], is_slippery=args["is_slippery"],
    render_mode=args["render_mode"])

simulator = FrozenLakeSimulator(env.P)

V_vit, Q_vit = value_iteration(
    simulator, args["gamma"], args["vit_thres"])
# global Q_vit_g = Q_vit
        
for state in range(simulator.num_states):
    logging.warning(f"\n-> state = {state}")
    logging.warning(f"V[state] = {V_vit[state]:0.4f}")
    for action in range(simulator.num_actions):
        logging.warning(f"Q[state][action] = {Q_vit[state][action]:0.4f}")
    logging.warning(f"best_action={np.argmax(Q_vit[state])}")
    
manager = mp.Manager()
ep_reward_list = manager.list()
Q_mcts_list = manager.list()

def run_trial(i_trial, Q_vit, args):

    random.seed(10000+i_trial)
    np.random.seed(10000+i_trial)

#     env = FrozenLakeCustom(
#         map_name=args["map_name"], is_slippery=args["is_slippery"],
#         render_mode=args["render_mode"])

#     simulator = FrozenLakeSimulator(env.P)

    Q_mcts, ep_reward = run_mcts_trial(env, simulator, Q_vit, i_trial, args)

    ep_reward_list.append(ep_reward)
    Q_mcts_list.append(Q_mcts)

In [3]:
print(f"num_trials = {args['num_trials']}")
print(f"mcts_max_its = {args['mcts_max_iterations']}")

Q_mcts_dict = defaultdict()

hparam_ucb_scale_ary = np.arange(20, 40, 2)

hparam_haver_var_ary = np.arange(0.0, 2, 0.5)
# hparam_haver_var_ary = [0.0001]
hparam_haver_var_ary = [0.5, 1.0, 2.0, 0.1]
best_param = None
max_reward_mean = -np.inf
for hparam_haver_var in hparam_haver_var_ary:
    print(f"\n-> hparam_haver_var = {hparam_haver_var}")
    args["hparam_haver_var"] = hparam_haver_var
    for hparam_ucb_scale in hparam_ucb_scale_ary:
        print(f"hparam_ucb_scale = {hparam_ucb_scale}")
        args["hparam_ucb_scale"] = hparam_ucb_scale
        
        start_time = time.time()
        
        pool = mp.Pool()
        pool.starmap(run_trial, [(i, Q_vit, args) for i in range(args["num_trials"])])

        reward_mean = np.mean(ep_reward_list)
        reward_std = np.std(ep_reward_list, ddof=1)
        print(f"reward = {reward_mean:.2f} +/- {reward_std:.2f}")

        # Q_mcts_dict[f"{hparam_haver_var}"] = copy.deepcopy(Q_mcts_list)

        if reward_mean > max_reward_mean:
            max_reward_mean = reward_mean 
            best_param = hparam_haver_var

        ep_reward_list[:] = []
        Q_mcts_list[:] = []

        end_time = time.time()
        print(f"it takes {end_time-start_time:0.4f}")

num_trials = 20
mcts_max_its = 2000

-> hparam_haver_var = 0.5
hparam_ucb_scale = 20
reward = -5.00 +/- 1.81
it takes 35.3906
hparam_ucb_scale = 22
reward = -5.05 +/- 1.88
it takes 40.1780
hparam_ucb_scale = 24
reward = -4.10 +/- 0.31
it takes 27.3624
hparam_ucb_scale = 26
reward = -4.55 +/- 0.83
it takes 32.5576
hparam_ucb_scale = 28
reward = -4.20 +/- 0.52
it takes 29.4409
hparam_ucb_scale = 30
reward = -4.20 +/- 0.70
it takes 29.1914
hparam_ucb_scale = 32
reward = -4.05 +/- 0.22
it takes 28.1405
hparam_ucb_scale = 34
reward = -4.10 +/- 0.45
it takes 28.5213
hparam_ucb_scale = 36
reward = -4.15 +/- 0.49
it takes 28.0269
hparam_ucb_scale = 38
reward = -4.10 +/- 0.31
it takes 28.0451

-> hparam_haver_var = 1.0
hparam_ucb_scale = 20
reward = -5.00 +/- 1.41
it takes 31.8442
hparam_ucb_scale = 22
reward = -4.50 +/- 1.24
it takes 33.3400
hparam_ucb_scale = 24
reward = -4.65 +/- 0.93
it takes 32.4996
hparam_ucb_scale = 26
reward = -4.20 +/- 0.41
it takes 28.4754
hparam_ucb_scale = 28
reward

In [4]:
Q_mcts_avg = defaultdict(lambda: np.zeros(simulator.num_actions))
for i_trial, Q_mcts in enumerate(Q_mcts_dict[f"{best_param}"]):
    for s in range(simulator.num_states):
        Q_mcts_avg[s] = (1-1/(i_trial+1))*Q_mcts_avg[s] + 1/(i_trial+1)*Q_mcts[s]
    
for state in range(simulator.num_states):
    print(f"\n-> state = {state}")
    print(f"V[state] = {np.max(Q_mcts_avg[state]):0.4f} | {np.max(Q_vit[state]):0.4f}")
    for action in range(simulator.num_actions):
        print(f"Q[state][action] = {Q_mcts_avg[state][action]:0.4f} | {Q_vit[state][action]:0.4f}")
    print(f"best_action = {np.argmax(Q_mcts_avg[state])} | {np.argmax(Q_vit[state])}")

KeyError: '0.1'